In [1]:
### Benchmarks for the 'generate_moves' function

In [1]:
from guard_towers import Board, BOARD_SIZE, Piece
from time import time
import pandas as pd

In [2]:
from timeit import timeit
from guard_towers import Board, Piece, BOARD_SIZE   # adjust import
from typing import Dict

def empty_board() -> Board:
    """Start with a completely blank board (no pieces)."""
    b = Board()
    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            b.grid[y][x] = None
    return b

def board_from_dict(pieces: Dict[str, Piece]) -> Board:
    """Create a board from a {square: Piece(...)} mapping."""
    b = empty_board()
    for sq, pc in pieces.items():
        b.place(sq, pc)
    return b

def time_generate_moves(board: Board, player: str, repeats: int = 10_000) -> float:
    """Return time (seconds) to call generate_moves `repeats` times."""
    return timeit(lambda: board.generate_moves(player), number=repeats)


In [3]:
initial_board = Board()
print("Blue to move:", time_generate_moves(initial_board, 'b'))
print("Red  to move:", time_generate_moves(initial_board, 'r'))


Blue to move: 0.1999410829739645
Red  to move: 0.18289470800664276


In [4]:
mid_game_pieces = {
    # Blue
    'D3': Piece('b', 'guard'),
    'E3': Piece('b', 'tower', 3),
    'B4': Piece('b', 'tower', 2),
    'G1': Piece('b', 'tower', 1),

    # Red
    'D6': Piece('r', 'guard'),
    'C5': Piece('r', 'tower', 2),
    'F6': Piece('r', 'tower', 4),
    'A7': Piece('r', 'tower', 1),
}
mid_game_board = board_from_dict(mid_game_pieces)

print("Blue to move:", time_generate_moves(mid_game_board, 'b'))
print("Red  to move:", time_generate_moves(mid_game_board, 'r'))


Blue to move: 0.3186817920068279
Red  to move: 0.27541312493849546


In [5]:
end_game_pieces = {
    # Blue
    'D2': Piece('b', 'guard'),
    'D3': Piece('b', 'tower', 4),

    # Red
    'D7': Piece('r', 'guard'),
    'D5': Piece('r', 'tower', 3),
}
end_game_board = board_from_dict(end_game_pieces)

print("Blue to move:", time_generate_moves(end_game_board, 'b'))
print("Red  to move:", time_generate_moves(end_game_board, 'r'))


Blue to move: 0.2349814169574529
Red  to move: 0.21963212499395013


### Benchmark eval function

In [6]:
import time
from guard_towers import fen_to_board
from evaluation import evaluate

def benchmark_evaluate(fen: str, player: str, iterations: int = 10_000):
    board, player = fen_to_board(fen)

    # Time the evaluation function over multiple iterations
    start = time.time()
    for _ in range(iterations):
        evaluate(board, player)
    end = time.time()

    total_time = end - start
    avg_time = total_time / iterations
    print(f"Evaluated {iterations} times in {total_time:.4f} seconds.")
    print(f"Average time per call: {avg_time:.8f} seconds.")

# Example usage, random board pos
benchmark_evaluate('r3RG5/r16/b16/7/7/7/3BG3 r', player='r')  # replace with your own FEN string

Evaluated 10000 times in 0.0111 seconds.
Average time per call: 0.00000111 seconds.


In [8]:
import time
from guard_towers import fen_to_board
from evaluation import find_best_move, transposition_table, pv_table

def benchmark_find_best_move_avg(fen: str, player: str, depths: list, runs_per_depth: int = 10):
    board_init, _ = fen_to_board(fen)

    for depth in depths:
        total_time = 0.0
        last_move = None
        for _ in range(runs_per_depth):
            board = board_init.copy()  # fresh board each time

            transposition_table.clear()
            pv_table.clear()
            
            start = time.perf_counter()
            move = find_best_move(board, player, base_depth=depth)
            end = time.perf_counter()
            total_time += (end - start)
            last_move = move  # just for display
        avg_time = total_time / runs_per_depth
        print(f"Depth {depth}: Avg time = {avg_time:.4f}s over {runs_per_depth} runs — Last move: {last_move}")

benchmark_find_best_move_avg('r1r11RG1r1r1/2r11r12/3r13/7/3b13/2b11b12/b1b11BG1b1b1 r', player='r', depths=[1, 2, 3, 4, 5, 6, 7, 8, 9])

Depth 1: Avg time = 0.0006s over 10 runs — Last move: D7-E7-1
Depth 2: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 3: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 4: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 5: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 6: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 7: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 8: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
Depth 9: Avg time = 0.0005s over 10 runs — Last move: D7-E7-1
